# Activity 9 : Machine Learning - Part 3 : Image Classification

In this part, we will build a machine learning model that's capable of differentiating between different flavors UHT drinkable yoguht, UHT milk, or sparkling water (depending on your room)

## Step 0 : Make a copy of this notebook

Before we begin, make a copy of this notebook first.

## Step 1 : Config Colab for ML usage

Config colab for ML usage, as done in the tutorial

check if configuration was done correctly by running the cells below

In [35]:
!nvidia-smi

Wed Mar 29 04:10:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    30W /  70W |   2431MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [36]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Step 2 : Mount Google Drive

Mount your Google Drive with this colab VM to use images you've uploaded prior

In [37]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Step 3 : Prepare data for model training

Run the code below to prepare your data for training

**Do not modify the code!**

In [63]:
train_img_path = f'/content/drive/My Drive/CEE_Act9-3/train'

In [64]:
from torchvision.datasets import ImageFolder
from torchvision import transforms

transform = transforms.Compose([
    transforms.ToTensor()
])

dataset_main = ImageFolder(train_img_path, transform=transform)
dataset_tmp = torch.utils.data.random_split(dataset_main, [0.9, 0.1])

dataset_train = dataset_tmp[0]
dataset_val = dataset_tmp[1]

In [65]:
from torch.utils.data import DataLoader

batch_size = 32             # If you get GPU memory error, reduce this value by half at a time

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=False, pin_memory=True)

## Step 4 : Train the model

Run the code below to train your machine learning model, this will take a few minutes

**Do not modify the code!**

In [66]:
from torch import nn
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

class ImageModel(nn.Module):
    def __init__(self):
        super(ImageModel, self).__init__()

        self.efficientnet = efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
        self.preprocess = EfficientNet_B0_Weights.DEFAULT.transforms()
        for param in self.efficientnet.parameters():
            param.requires_grad = False
        self.efficientnet.classifier[1] = nn.Linear(1280, 4)

    def forward(self, x):
        x = self.preprocess(x)
        x = self.efficientnet(x)

        return x

In [67]:
import copy, time

def train_model(model, dataloader_train, dataloader_val, num_epoch, lossfn, optimizer, save_path=None):
    best_model_weight = copy.deepcopy(model.state_dict())
    best_loss = float('inf')
    val_acc_his = []

    for epoch in range(num_epoch):
        print(f'epoch {epoch+1} / {num_epoch}')
        start_time = time.time()
        model.train()

        running_loss_train = 0.0
        running_loss_val = 0.0

        for image, label in dataloader_train:
            image = image.to(device)
            label = label.to(device)

            optimizer.zero_grad()

            output = model(image)
            loss = lossfn(output, label)

            loss.backward()
            optimizer.step()

            running_loss_train += loss.item() * image.size(0)

        epoch_loss_train = running_loss_train / (len(dataloader_train.dataset) * 5)

        model.eval()

        for image, label in dataloader_val:
            image = image.to(device)
            label = label.to(device)

            output = model(image)
            loss = lossfn(output, label)

            running_loss_val += loss.item() * image.size(0)
        
        epoch_loss_val = running_loss_val / len(dataloader_val.dataset)
        val_acc_his.append(epoch_loss_val)

        print(f'epoch {epoch+1} training completed ; training loss: {epoch_loss_train:.6f} ; validation loss: {epoch_loss_val:.6f}')

        if epoch_loss_val < best_loss:
            print('validation loss improved, saving model')
            best_loss = epoch_loss_val
            best_model_weight = copy.deepcopy(model.state_dict())
            if save_path: torch.save(model.state_dict(), save_path)
        else:
            print(f'validation loss did not improved from {best_loss:.6f}')

        print(f'epoch {epoch+1} completed in {time.time() - start_time:.4f} seconds')
        print()

    model.load_state_dict(best_model_weight)
    return model, val_acc_his

In [68]:
model = ImageModel()
model.to(device)

lossfn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [69]:
model_save_path = f'/content/drive/My Drive/CEE_Act9-3/model-3.pt'

In [ ]:
epoch_cnt = 20
model, val_acc_his = train_model(model, train_loader, val_loader, epoch_cnt, lossfn, optimizer, model_save_path)

epoch 1 / 20
epoch 1 training completed ; training loss: 0.194050 ; validation loss: 0.548956
validation loss improved, saving model
epoch 1 completed in 192.4167 seconds

epoch 2 / 20
epoch 2 training completed ; training loss: 0.096869 ; validation loss: 0.266732
validation loss improved, saving model
epoch 2 completed in 45.9934 seconds

epoch 3 / 20


## Step 5 : Test the model

Now, let's test the model with another set of data!

**Do not modify the code except the one marked with comments!**

In [47]:
!git clone 'https://github.com/leo2tigers/compengess2023-act9.git'

fatal: destination path 'compengess2023-act9' already exists and is not an empty directory.


In [60]:
# Enter your room no. (401, 404, 502) here
room_no = 502

In [61]:
import os
from torch.utils.data import Dataset
from torchvision.datasets.folder import default_loader

class ImageDataset(Dataset):
    def __init__(self, paths, transform=None):
        self.paths = paths
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        image = default_loader(f'compengess2023-act9/test/{room_no}/{self.paths[index]}')
        if self.transform is not None:
            image = self.transform(image)
        return image

def test_model(img_paths):
    image = ImageDataset(img_paths, transform=transform)
    loader = torch.utils.data.DataLoader(image, batch_size=1)

    all_predictions = []
    model.eval()
    with torch.no_grad():
        for img in loader:
            predictions = list(nn.Softmax(dim=1)(model(img.to(device))).cpu().numpy())
            for prediction in predictions:
                all_predictions.append(prediction)

    return all_predictions

def detail_result(label_list, filename_and_pred):
    print(filename_and_pred[0])
    for i in range(len(label_list)):
        print(f'{label_list[i]}: {filename_and_pred[1][i]*100:.3f}%')
    print()

In [62]:
label_list = sorted(os.listdir(f"/content/drive/My Drive/CEE_Act9-3/train"))
file_list = os.listdir(f'./compengess2023-act9/test/{room_no}')
res = list(zip(file_list, test_model(file_list)))
for entry in res: detail_result(label_list, entry)

IMG_20230329_091237.jpg
sparkling_colalime: 13.628%
sparkling_honeyyuzu: 44.985%
sparkling_jplemon: 35.616%
sparkling_whitepeach: 5.772%

IMG_20230329_090650.jpg
sparkling_colalime: 25.474%
sparkling_honeyyuzu: 3.094%
sparkling_jplemon: 68.503%
sparkling_whitepeach: 2.929%

IMG_20230328_233551.jpg
sparkling_colalime: 6.716%
sparkling_honeyyuzu: 33.557%
sparkling_jplemon: 58.950%
sparkling_whitepeach: 0.777%

IMG_20230328_233610.jpg
sparkling_colalime: 6.253%
sparkling_honeyyuzu: 5.475%
sparkling_jplemon: 87.308%
sparkling_whitepeach: 0.963%

IMG_20230329_090544.jpg
sparkling_colalime: 16.994%
sparkling_honeyyuzu: 2.015%
sparkling_jplemon: 80.759%
sparkling_whitepeach: 0.233%

IMG_20230328_233629.jpg
sparkling_colalime: 2.291%
sparkling_honeyyuzu: 66.909%
sparkling_jplemon: 30.649%
sparkling_whitepeach: 0.151%

IMG_20230329_090720.jpg
sparkling_colalime: 3.145%
sparkling_honeyyuzu: 1.874%
sparkling_jplemon: 94.420%
sparkling_whitepeach: 0.561%

IMG_20230329_091205.jpg
sparkling_colalime

In [ ]:
file_list

['IMG_20230329_091237.jpg',
 'IMG_20230329_090650.jpg',
 'IMG_20230328_233551.jpg',
 'IMG_20230328_233610.jpg',
 'IMG_20230329_090544.jpg',
 'IMG_20230328_233629.jpg',
 'IMG_20230329_090720.jpg',
 'IMG_20230329_091205.jpg',
 'IMG_20230328_233532.jpg',
 'IMG_20230329_091223.jpg',
 'IMG_20230329_090621.jpg',
 'IMG_20230329_091257.jpg']

**Please show the result from the cell above to an instructor or TA to be graded**

***- THIS IS THE END OF PART 3 -***